In [1]:
import torch
import gc
from dataloader import CustomImageDataset, DataLoader
from torchvision.transforms import Compose
import torchvision.transforms as transforms

transform = Compose([
    transforms.ToTensor(), # Scales data into [0,1]
    transforms.Lambda(lambda t: (t * 2) - 1) # Scale between [-1, 1]
])

dataset = CustomImageDataset(annotations_dir = '/home/pqbas/dl/detection/MNIST-ObjectDetection/data/mnist_detection/test/labels',
                             img_dir = '/home/pqbas/dl/detection/MNIST-ObjectDetection/data/mnist_detection/test/images',
                             data_transform = transform,
                             size=(448,448))

train_loader = DataLoader(dataset, batch_size=16, shuffle=True)


def clean_gpu():
    for i in range(10):    
        torch.cuda.empty_cache()
        gc.collect()

device = torch.device('cuda:0')

In [19]:
from model import YOLOv1
import torchvision.transforms as T
from torch import nn
from torch import optim
import numpy as np


# to plot the loss
loss_history = []

# clean gpu before wathever
clean_gpu()

# model
model = YOLOv1(S=7, depth=1).to(device)
model.train()

# optimizatino algorithm
mse = nn.MSELoss(reduction='sum')
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

#training loop
EPOCHS = 10
for epoch in range(EPOCHS):
    n_break = 0
    for idx, data in enumerate(train_loader):
        # get the inputs
        img, target = data
        img, target = img.to(device), target['one_obj'].to(device)

        # zero the parameters
        optimizer.zero_grad()
        
        # forward
        prediction = model(img)
        predict_confidence = torch.sigmoid(prediction[...,0])
        target_confidence = target.squeeze()

        # backward + optimize
        loss = mse(target_confidence, predict_confidence)
        loss.backward()
        optimizer.step()


        loss_history.append(loss.detach().item())

        # if idx == n_break:
        #     break

    print(f'Loss at {epoch}/{EPOCHS} is: {np.average(np.array(loss_history))}')




Loss at 0/10 is: 138.23953961569165
Loss at 1/10 is: 127.59218500530909
Loss at 2/10 is: 124.0501427827058
Loss at 3/10 is: 122.26870162903316
Loss at 4/10 is: 121.1739147004627
Loss at 5/10 is: 120.44005097283258
Loss at 6/10 is: 119.91102423635469
Loss at 7/10 is: 119.52486708807567
Loss at 8/10 is: 119.21416638894055
Loss at 9/10 is: 118.98168389456613


In [ ]:
    img = (img + 1)/2
    img_ = T.ToPILImage()(img[0])

    w = pred_box1[0,:,:,2]*448
    h = pred_box1[0,:,:,2]*448



In [5]:
import torch
import torch.nn as nn
from metrics import intersection_over_union
from model import YOLOv1

model = YOLOv1(B=2, C=10, S=7)

mse = nn.MSELoss(reduction='sum')

n_break = 0
for idx,(img, target) in enumerate(train_loader):
    
    one_obj = one_obj[...,0][...,None]
    
    prediction = model(img)

    # ========================== #
    #        Object Loss         #
    # ========================== #

    target_box = target[...,0:4]

    box1 = torch.sigmoid(prediction[...,0:4])
    box2 = torch.sigmoid(prediction[...,4:8])

    iou1 = intersection_over_union(box1, target_box)
    iou2 = intersection_over_union(box2, target_box)
    
    ious = torch.cat([iou1,iou2], dim=3)
    max, bestbox = torch.max(ious, dim=3)
    
    box = (bestbox[...,None]*box2 +  (1 - bestbox[...,None])*box1) * one_obj

    box[...,2:4] = torch.sqrt(box[...,2:4])
    target[...,2:4] = torch.sqrt(target_box[...,2:4])

    box_coordinates_loss = mse(box,target_box)
    
    # ========================== #
    #        Classf Loss         #
    # ========================== #

    pred_prob = prediction[...,10:21]*one_obj
    target_prob = target[...,4:15]*one_obj

    classf_loss = mse(pred_prob, target_prob)


    # ========================= #
    #        Noobj Loss         #
    # ========================= #

    noobj = 1 - one_obj
    noobj_pred_prob = prediction[...,10:21]*noobj
    noobj_target_prob = target[...,4:15]*noobj

    print(noobj_pred_prob.shape)
    print(noobj_target_prob.shape)

    mse(noobj_pred_prob, noobj_target_prob)


    if idx == n_break:
        break

NameError: name 'one_obj' is not defined

In [ ]:
import torch
import torch.nn as nn
from metrics import intersection_over_union
from model import YOLOv1

model = YOLOv1(B=2, C=10, S=7)

n_break = 0
for idx,(img, target) in enumerate(train_loader):
    
    one_obj = one_obj[...,0][...,None]
    
    prediction = model(img)

    target['']

    mse(noobj_pred_prob, noobj_target_prob)


    if idx == n_break:
        break
